In [ ]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

# Importing selenium webdriver 
from selenium import webdriver

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

# Importing requests
import requests

# Importing regex
import re

1. Write a python program which searches all the product under a particular product vertical
from www.amazon.in. The product verticals to be searched will be taken as input from user.
For e.g. If user input is ‘guitar’. Then search for guitars.

In [ ]:
driver = webdriver.Chrome(r"C:/Users/HP/Downloads/chromedriver_win32/chromedriver.exe")

url = "https://www.amazon.in/"
driver.get(url)
time.sleep(2)

# Asking the user to input the keywords he/she wants to search
user_inp = input('Enter the product you want to search : ')

# Locating search_bar by id
search_bar = driver.find_element_by_id("twotabsearchtextbox")
# clearing search_bar
search_bar.clear()
# Sending user input to search bar
search_bar.send_keys(user_inp)
# Locating search_button by xpath
search_button = driver.find_element_by_xpath('//div[@class="nav-search-submit nav-sprite"]/span/input')       
search_button.click()  

2. In the above question, now scrape the following details of each product listed in first 3 pages 
of your search results and save it in a data frame and csv. In case if any product vertical has 
less than 3 pages in search results then scrape all the products available under that product 
vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of 
Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" 
and “Product URL”. In case, if any of the details are missing for any of the product then 
replace it by “-“.

In [ ]:
start_page = 0
end_page = 3
urls = []
for page in range(start_page,end_page+1):
    try:
        page_urls = driver.find_elements_by_xpath('//a[@class="a-link-normal s-no-outline"]')
        
        # Appending all the urls on current page to urls list
        for url in page_urls:
            url = url.get_attribute('href')     # Scraping the url from webelement
            if url[0:4]=='http':                # Checking if the scraped data is a valid url or not
                urls.append(url)                # Appending the url to urls list
        print("Product urls of page {} has been scraped.".format(page+1))
        
        # Moving to next page
        nxt_button = driver.find_element_by_xpath('//li[@class="a-last"]/a')      # Locating the next_button which is active
        if nxt_button.text == 'Next→':                                            # Checking if the button located is next button
            nxt_button.click()                                                    # Clicking the next button
            time.sleep(6)                                                         # time delay of 5 seconds
        # If the current active button is not next button, the we will check if the next button is inactive or not    
        elif driver.find_element_by_xpath('//li[@class="a-disabled a-last"]/a').text == 'Next→':    
            print("No new pages exist. Breaking the loop")  # Printing message and breakinf loop if we have reached the last page
            break
            
    except StaleElementReferenceException as e:             # Handling StaleElement Exception   
        print("Stale Exception")
        next_page = nxt_button.get_attribute('href')        # Extracting the url of next page
        driver.get(next_page)                               # ReLoading

In [ ]:
prod_dict = {}
prod_dict['Brand']=[]
prod_dict['Name']=[]
prod_dict['Rating']=[]
prod_dict['No. of ratings']=[]
prod_dict['Price']=[]
prod_dict['Return/Exchange']=[]
prod_dict['Expected Delivery']=[] 
prod_dict['Availability']=[]
prod_dict['Other Details']=[]
prod_dict['URL']=[]

for url in urls[:4]:
    driver.get(url)                                                        # Loading the webpage by url
    print("Scraping URL = ", url)
    #time.sleep(2)
    
    try:
        brand = driver.find_element_by_xpath('//a[@id="bylineInfo"]')      # Extracting Brand from xpath
        prod_dict['Brand'].append(brand.text)
    except NoSuchElementException:
        prod_dict['Brand'].append('-')
    
    try:
        name = driver.find_element_by_xpath('//h1[@id="title"]/span')      # Extracting Name from xpath
        prod_dict['Name'].append(name.text)
    except NoSuchElementException:
        prod_dict['Name'].append('-')
    
    try:
        rating = driver.find_element_by_xpath('//span[@id="acrPopover"]')  # Extracting Ratings from xpath
        prod_dict['Rating'].append(rating.get_attribute("title"))
    except NoSuchElementException:
        prod_dict['Rating'].append('-')
    
    try:
        n_rating = driver.find_element_by_xpath('//a[@id="acrCustomerReviewLink"]/span')     # Extracting no. of Ratings from xpath
        prod_dict['No. of ratings'].append(n_rating.text)
    except NoSuchElementException:
        prod_dict['No. of ratings'].append('-')
    
    try:
        price = driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]')            # Extracting Price from xpath
        prod_dict['Price'].append(price.text)
    except NoSuchElementException:
        prod_dict['Price'].append('-')
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        ret = driver.find_element_by_xpath('//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        prod_dict['Return/Exchange'].append(ret.text)
    except NoSuchElementException:
        prod_dict['Return/Exchange'].append('-')
    try:
        delivry = driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b')         # Extracting Expected Delivery from xpath
        prod_dict['Expected Delivery'].append(delivry.text)
    except NoSuchElementException:
        prod_dict['Expected Delivery'].append('-')
    
    try:
        avl = driver.find_element_by_xpath('//div[@id="availability"]/span')                # Extracting Availability from xpath
        prod_dict['Availability'].append(avl.text)
    except NoSuchElementException:
        prod_dict['Availability'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        dtls = driver.find_element_by_xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]')
        prod_dict['Other Details'].append('  ||  '.join(dtls.text.split('\n')))
    except NoSuchElementException:
        prod_dict['Other Details'].append('-')
    
    prod_dict['URL'].append(url)                                                            # Saving url
    time.sleep(2)

In [ ]:
prod_df = pd.DataFrame.from_dict(prod_dict)
prod_df

In [ ]:
# Saving data to csv
prod_df.to_csv('Amazon_{}.csv'.format(user_inp))

3. Write a python program to access the search bar and search button on images.google.com and 
scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [ ]:
driver = webdriver.Chrome(r"C:/Users/HP/Downloads/chromedriver_win32/chromedriver.exe")

url = "https://images.google.com/"
driver.get(url)
time.sleep(2)


search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')    # Finding the search bar using it's xpath
search_bar.send_keys("fruits")       # Inputing "fruits" keyword to search images
search_button = driver.find_element_by_xpath('//*[@id="sbtc"]/button')    # Finding the xpath of search button
search_button.click()        # Clicking the search button


# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")
    
images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
            
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open(r"F:\Data Science Internship\Fruit"+str(i)+".jpg", "wb")
    file.write(response.content)

4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on 
www.flipkart.com and scrape following details for all the search results displayed on 1st page. 
Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, 
“Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display 
Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. 
Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe 
and CSV.

In [ ]:
driver = webdriver.Chrome(r"C:/Users/HP/Downloads/chromedriver_win32/chromedriver.exe")

# Asking the user to input the keywords he/she wants to search
prod = input(" Enter the name of the mobile phone you want to search : ")
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(2)


try:
    login_X_button = driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]')                      # Button to close login popup
    login_X_button.click()
except NoSuchElementException : 
    print("No Login page")
    
search_bar = driver.find_element_by_xpath('//*[@id="container"]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')    # Finding the search bar using it's xpath
search_bar.clear()               # Clearing the search bar
search_bar.send_keys(prod)       # Inputing keyword to search
search_button = driver.find_element_by_xpath('//button[@class="L0Z3Pu"]')    # Finding the xpath of search button
search_button.click()        # Clicking the search button

In [ ]:
# Fetching urls of phones coming on 1st page
flip_urls = []
urls = driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')
for url in urls:
    flip_urls.append(url.get_attribute("href"))

In [ ]:
len(flip_urls)

In [ ]:
flip_dict = {}
flip_dict["Brand"] = []
flip_dict["Smartphone"] = []
flip_dict["Colour"] = []
flip_dict["RAM"] = []
flip_dict["Storage(ROM)"] = []
flip_dict["Primary Camera"] = []
flip_dict["Secondary Camera"] = []
flip_dict["Display Size"] = []
flip_dict["Display Resolution"] = []
flip_dict["Processor"] = []
flip_dict["Processor Cores"] = []
flip_dict["Battery Capacity"] = []
flip_dict["Battery Type"] = []
flip_dict["Price"] = []
flip_dict["URL"] = []

In [ ]:
# Scraping data from each url
for url in flip_urls:
    driver.get(url)    # Saving url                                                     
    print("Scraping URL = ", url)
    flip_dict['URL'].append(url)                                                          # Loading the webpage by url
    time.sleep(2)
    
    try:
        read_more = driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]')     # Button for expanding the specs
        read_more.click()
    except NoSuchElementException:
        print("Exception Occured. Moving to next page")
    
    try:
        brand = driver.find_element_by_xpath('//span[@class="B_NuCI"]')      # Extracting Brand from xpath
        flip_dict["Brand"].append(brand.text.split()[0])
    except NoSuchElementException:
        flip_dict['Brand'].append('-')
        
    try:
        price = driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')      # Extracting Brand from xpath
        flip_dict['Price'].append(price.text)
    except NoSuchElementException:
        flip_dict['Price'].append('-')
        
    try:
        name = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')      # Extracting Brand from xpath
        flip_dict['Smartphone'].append(name.text)
    except NoSuchElementException:
        flip_dict['Smartphone'].append('-')
    
    try:
        color = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      # Extracting Name from xpath
        flip_dict['Colour'].append(color.text)
    except NoSuchElementException:
        flip_dict['Colour'].append('-')
    
    try:
        disp_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_chk.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  # Extracting Ratings from xpath
        flip_dict['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        flip_dict['Display Size'].append('-')
    
    try:
        disp_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_chk.text != "Display Features" : raise NoSuchElementException
        disp_res = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[2]/td[2]/ul/li')     # Extracting no. of Ratings from xpath
        flip_dict['Display Resolution'].append(disp_res.text)
    except NoSuchElementException:
        flip_dict['Display Resolution'].append('-')
    
    try:
        pro_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
        if pro_chk.text != "Processor Type" : raise NoSuchElementException
        processor = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')   # Extracting Price from xpath
        flip_dict['Processor'].append(processor.text)
    except NoSuchElementException:
        flip_dict['Processor'].append('-')
    
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        core_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[1]')
        if core_chk.text != "Processor Core" :
            core_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
            if core_chk.text != "Processor Core" : 
                raise NoSuchElementException
            else :
                cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')
        else :
            cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[2]/ul/li')
        flip_dict['Processor Cores'].append(cores.text)
    except NoSuchElementException:
        flip_dict['Processor Cores'].append('-')
    
    try:
        rom = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')         # Extracting Expected Delivery from xpath
        flip_dict['Storage(ROM)'].append(rom.text)
    except NoSuchElementException:
        flip_dict['Storage(ROM)'].append('-')
    
    try:
        ram = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['RAM'].append(ram.text)
    except NoSuchElementException:
        flip_dict['RAM'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        pri_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        flip_dict['Primary Camera'].append(pri_cam.text)
    except NoSuchElementException:
        flip_dict['Primary Camera'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        cam_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if cam_chk != "Secondary Camera" : 
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        flip_dict['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        flip_dict['Secondary Camera'].append('-')
        
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
            bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['Battery Capacity'].append(bat_cap.text)
    except NoSuchElementException:
        flip_dict['Battery Capacity'].append('-')
    
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr[2]/td[1]')
                if bat_chk.text != "Battery Type" : raise NoSuchElementException
                bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr[2]/td[2]/ul/li')
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr[2]/td[1]')
                if bat_chk.text != "Battery Type" : raise NoSuchElementException
                bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr[2]/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[1]')
            if bat_chk.text != "Battery Type" : raise NoSuchElementException
            bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['Battery Type'].append(bat_typ.text)
    except NoSuchElementException:
        flip_dict['Battery Type'].append('-')
            

In [ ]:
print(len(flip_dict["Brand"]), len(flip_dict["Smartphone"]), len(flip_dict["Processor"]), len(flip_dict["Price"]), len(flip_dict['URL']))    

In [ ]:
flip_df = pd.DataFrame.from_dict(flip_dict)
flip_df

5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [ ]:
driver = webdriver.Chrome(r"C:/Users/HP/Downloads/chromedriver_win32/chromedriver.exe")

url = "https://www.google.co.in/maps"
driver.get(url)
time.sleep(2)


city = input('Enter City Name : ')                                         # Enter city to be searched
search = driver.find_element_by_id("searchboxinput")                       # locating search bar
search.clear()                                                             # clearing search bar
time.sleep(2)
search.send_keys(city)                                                     # entering values in search bar
button = driver.find_element_by_id("searchbox-searchbutton")               # locating search button
button.click()                                                             # clicking search button
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 –
September 20) from trak.in.

In [ ]:
driver = webdriver.Chrome(r"C:/Users/HP/Downloads/chromedriver_win32/chromedriver.exe")

url = "https://trak.in/"
driver.get(url)
time.sleep(2)

In [ ]:
button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(button)

In [ ]:
fund_dict = {}
fund_dict['Date'] = []
fund_dict['Startup Name'] = []
fund_dict['Industry/Vertical'] = []
fund_dict['Sub-Vertical'] = []
fund_dict['Location'] = []
fund_dict['Investor'] = []
fund_dict['Investment Type'] = []
fund_dict['Amount(in USD)'] = []

for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()

    # Date
    dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_dict['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_dict['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_dict['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_dict['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_dict['Location'].append(l.text)
    
    # Investor
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_dict['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_dict['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_dict['Amount(in USD)'].append(a.text)
    
fund_df = pd.DataFrame(fund_dict)
fund_df

In [ ]:
fund_df.to_csv("Indian Startups_Q2_2020.csv")

7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [ ]:
driver = webdriver.Chrome(r"C:/Users/HP/Downloads/chromedriver_win32/chromedriver.exe")

url = "https://www.digit.in/"
driver.get(url)
time.sleep(2)

In [ ]:
#Clicking on top 10 option
top10=driver.find_element_by_xpath("//div[@class='menu']/ul/li[4]/a")
top10.click()

#Clicking on best gaming laptops option from top10
best_gl=driver.find_element_by_xpath("//div[@class='listing_container']/ul/li[26]/a")
best_gl.click()

#Specifying the url of the webpage to be scraped
url="https://www.digit.in/top-products/best-gaming-laptops-40.html"
driver.get(url)

#Extracting the tags having the laptop name
name=driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")

#Extracting the text from the tags
prod_name=[]  #Empty list

#As we need to scrap data for all the products, we are running a for loop for extracting all data
for i in name:
    prod_name.append(i.text)

#Extracting the tags having the OS type
OS_type=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[1]/div/div")

#Extracting the text from the tags
OS=[]  #Empty list

#As we need to scrap data for all the products, we are running a for loop for extracting all data
for i in OS_type:
    OS.append(i.text)

#Extracting the tags having display details
display=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[2]/div/div")

#Extracting the text from the tags
display_specs=[]  #Empty list

#As we need to scrap data for all the products, we are running a for loop for extracting all data
for i in display:
    display_specs.append(i.text)

#Extracting the tags having processor details
processor=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[3]/div/div")

#Extracting the text from the tags
processor_specs=[]  #Empty list

#As we need to scrap data for all the products, we are running a for loop for extracting all data
for i in processor:
    processor_specs.append(i.text)

#Extracting the tags having memory specs

#List of specification names
memory=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[1]")

#Value of the specifications
memory_specs=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")

#Now we will separate HDD and RAM text from memory specs tags
HDD=[]
RAM=[]  #Empty lists
for i in range(len(memory)):
    if memory[i].text=='Memory':
        HDD.append(memory_specs[i].text.split('/')[0])
        RAM.append(memory_specs[i].text.split('/')[1])
    else:
        HDD.append('Not Available')
        RAM.append('Not Available')

#Extracting the tags having weight

#List of specifications name
weights=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")

#Value of the specifications
weights_specs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")

#Now we will separate weight text from tags
weight=[] #Empty list
for i in range(len(weights)):
    if weights[i].text=='Weight':
        weight.append(weights_specs[i].text)

#Extracting the tags having dimensions

#List of specifications name
dims=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")

#Value of the specifications
dims_specs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")

#Now we will separate dimensions text from tags
dimension=[] #Empty list
for i in range(len(dims)):
    if dims[i].text=='Dimension':
        dimension.append(dims_specs[i].text)

#Extracting the tags having Graphics Processor

#List of specifications name
GPs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")

#Value of the specifications
GPs_specs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")

#Now we will separate GPs text from tags
GPU=[] #Empty list
for i in range(len(GPs)):
    if GPs[i].text=='Graphics Processor':
        GPU.append(GPs_specs[i].text)
        
#Extracting the tags having the price

#As there are some prices in the main url, we will go to full specs and scrape the prices
#First we will extract the urls of all laptop's full specs
full_specs=[]   #Empty list
urls=driver.find_elements_by_xpath("//div[@class='full-specs']/span")

#Running a for loop for extraction of text from tags
for i in urls:
    if i.get_attribute('data-href'):
        full_specs.append(i.get_attribute('data-href'))
        
#Now we will extract price by iterating full_specs
Price=[]  #Empty list
for i in full_specs:
    driver.get(i)
    try:
        prices=driver.find_element_by_xpath("//div[@class='Block-price']/b")  #Getting price tags
        Price.append(prices.text)  #Extracting text
    except NoSuchElementException as e:   #Running an exception if the price is not available
         Price.append("Not Available")    #Message to be printed in places where the price is n

In [ ]:
#Make data frame
df=pd.DataFrame({"Product Name":prod_name,
                "price":Price,
                "OS":OS,
                "Display":display_specs,
                "HDD":HDD,
                 "RAM":RAM,
                "processor":processor_specs,
                "weight":weight,
                "Dimension":dimension,
                "Graphical processor":GPU})
df

In [ ]:
df.to_csv("Gaming laptops_digit.csv")

8. Write a python program to scrape the details for all billionaires from www.forbes.com. 
Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, 
“Industry”.

In [ ]:
driver = webdriver.Chrome(r"C:/Users/HP/Downloads/chromedriver_win32/chromedriver.exe")

url = "https://www.forbes.com/?sh=69e6b8c92254"
driver.get(url)
time.sleep(2)

In [ ]:
#clicking the explore button
button = driver.find_element_by_xpath("//button[@class='icon--hamburger']")
button.click()
time.sleep(1)

#select billionaire  
billioners = driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]")
billioners.click()
time.sleep(1)

#select world billionaire  
world_billioners= driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a")
world_billioners.click()
time.sleep(1)


#make empty listes
Rank = [] 
Person_Name = [] 
total_net_worth = [] 
Age = []
country_of_citizenship = [] 
Source = []
industry = []


while(True):
    #scraping rank of billionaire 
    rank= driver.find_elements_by_xpath("//div[@class='rank']")
    for i in rank:
        Rank.append(i.text)
    time.sleep(1)
    
    #scraping name of billionaire 
    name= driver.find_elements_by_xpath("//div[@class='personName']//div")
    for i in name:
        Person_Name.append(i.text)
    time.sleep(1)
    
     #scraping Age of billionaire 
    age= driver.find_elements_by_xpath("//div[@class='age']//div")
    for i in age:
        Age.append(i.text)   
    time.sleep(1)
    
    #scraping citizenship of billionaire     
    citizenship= driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for i in citizenship:
        country_of_citizenship.append(i.text)
    time.sleep(1)
    
    #scraping source of income
    source= driver.find_elements_by_xpath("//div[@class='source']")
    for i in source:
        Source.append(i.text)    
    time.sleep(1)
        
    #scraping Age of billionaire 
    industries= driver.find_elements_by_xpath("//div[@class='category']//div")
    for i in industries:
        industry.append(i.text)
        
    #scraping net_worth of billionaire 
    net_worth= driver.find_elements_by_xpath("//div[@class='netWorth']//div[1]")
    for i in net_worth:
        total_net_worth.append(i.text)
    time.sleep(1)    
        
    try:
        next_button = driver.find_element_by_xpath("//button[@class='pagination-btn pagination-btn--next ']")
        next_button.click()
    except:
        break  

        
Net_Worth = []        
for i in range(0,len(total_net_worth),2):
        Net_Worth.append(total_net_worth[i])
    

time.sleep(2)        
#creating dataframe
df=pd.DataFrame({'Rank':Rank,
                'Name':Person_Name,
                'Net Worth':Net_Worth,
                'Age':Age,
                'Country of Citizenship':country_of_citizenship,
                'Source':Source,
                'Industry':industry})
df

9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [ ]:
driver = webdriver.Chrome(r"C:/Users/HP/Downloads/chromedriver_win32/chromedriver.exe")

url = "https://www.youtube.com/"
driver.get(url)
time.sleep(2)

In [ ]:
#finding element for search bar
search_bar = driver.find_element_by_id('search')
search_bar.send_keys("GOT")  #entering Video name
time.sleep(1)

#clicking on search button
search_btn = driver.find_element_by_id("search-icon-legacy")  
search_btn.click()
time.sleep(1)

#clicking on first video
link_click = driver.find_element_by_xpath("//yt-formatted-string[@class ='style-scope ytd-video-renderer']")
link_click.click()

In [ ]:
# 1000 time we scroll down by 10000 in order to generate more Comments
for _ in range(1000):
    driver.execute_script("window.scrollBy(0,10000)")

In [ ]:
comments = []
comment_time = []
Time = []
Likes = []
No_of_Likes = []

#scrape comments
cm = driver.find_elements_by_id("content-text")
for i in cm:
    if i.text is None:
        comments.append("--")
    else:
        comments.append(i.text)
time.sleep(5)


# scrape time when comment was posted
tm = driver.find_elements_by_xpath("//a[contains(text(),'ago')]")
for i in tm:
    Time.append(i.text)

for i in range(0,len(Time),2):
    comment_time.append(Time[i])
time.sleep(5)
    
# scrape the comment likes
like = driver.find_elements_by_xpath("//span[@class='style-scope ytd-comment-action-buttons-renderer']")
for i in like:
    Likes.append(i.text)
    
for i in range(1,len(Likes),2):
    No_of_Likes.append(Likes[i])
    
    
time.sleep(2)
#creating dataframe
df=pd.DataFrame({'Comments':comments,
                'Time':comment_time,
                'Likes':No_of_Likes})
df

10. Write a python program to scrape a data for all available Hostels from 
https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, 
distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms 
from price, facilities and property description.

In [ ]:
driver = webdriver.Chrome(r"C:/Users/HP/Downloads/chromedriver_win32/chromedriver.exe")

url = "https://www.hostelworld.com/"
driver.get(url)
time.sleep(2)

In [ ]:
#locating the location search bar
search_loc = driver.find_element_by_id('search-input-field')
# write Lonodn in search bar
search_loc.send_keys("London")
time.sleep(2)

#select london
london = driver.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]/div')
london.click()
time.sleep(2)


# do click on search button
search_btn = driver.find_element_by_id('search-button')
search_btn.click()

In [ ]:
Hostel_Name = []
Distance = []
overall_review = []
total_reviews = []
facilities = []
price = []
Rating = []
property_description = []

while(True):
    
    # Hostel name
    names = driver.find_elements_by_xpath('//h2[@class="title title-6"]')
    for name in names:
        Hostel_Name.append(name.text)
    time.sleep(2)
        
    # Distance from city centre
    dis = driver.find_elements_by_xpath('//span[@class="description"]')
    for d in dis:
        Distance.append(d.text)
    time.sleep(2)
        
    # Overall Review    
    review = driver.find_elements_by_xpath('//div[@class="keyword"]//span')
    for r in review:
        overall_review.append(r.text)
    time.sleep(2)
    
    # Total No of reviews     
    t_review = driver.find_elements_by_xpath('//div[@class="reviews"]')
    for tr in t_review:
        total_reviews.append(tr.text)
    time.sleep(2)
    
    # facilities
    service = driver.find_elements_by_xpath('//div[@class="facilities-label facilities"]')
    for s in service:
        facilities.append(s.text)
    time.sleep(2)
    
    # Prices    
    prices = driver.find_elements_by_xpath('//div[@class="price-col"]')
    for p in prices:
        price.append(p.text)
    time.sleep(2)    
        
    try:
        next_button = driver.find_element_by_xpath('//div[@class="pagination-item pagination-next"]')
        next_button.click()
    except:
        break
        
time.sleep(4)        
        
# Separate  Privates_From price  and  Dorms_From price
Privates_From = []
for i in range(0,len(price),2):
    Privates_From.append(price[i])
time.sleep(2)

Dorms_From = []
for i in range(1,len(price),2):
    Dorms_From.append(price[i])  

In [ ]:
# scrape Hostels URL
hostel_url = []

while(True):
    urls = driver.find_elements_by_xpath('//h2[@class="title title-6"]//a')
    for url in urls:
        hostel_url.append(url.get_attribute("href"))
    time.sleep(2)    
        
    try:
        next_button = driver.find_element_by_xpath('//div[@class="pagination-item pagination-next"]')
        next_button.click()
    except:
        break


        
Rate = []
for page in hostel_url:
    driver.get(page)
    
    # Rating
    try:
        ratings = driver.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/section/div[6]/div/div[1]/div[1]/div[1]')
        Rate.append(ratings.text)
    except NoSuchElementException:
        Rate.append("No Rating")  
    time.sleep(2)
    
    
    # Property Description
    try:
        pd = driver.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/section/div[6]/div/div[2]/div[2]/div/div[2]')
        property_description.append(pd.text)
    except NoSuchElementException:
        property_description.append("No Description")  

    
time.sleep(2)        
# remove extra data from Rating     
all_text = []
for i in Rate:
    all_text.append(i.split())
time.sleep(2)

for i in all_text:
    Rating.append(i[0]) 

In [ ]:
# Scraping the data by using dataframe
df = pd.DataFrame({'Hostel Name':Hostel_Name,
                'Distance from city centre':Distance,
                'Overall Review':overall_review[3:],
                'Total Reviews':total_reviews,
                'Facilities':facilities,
                'Privates From Price':Privates_From,
                'Dorms From Price':Dorms_From,
                'Rating':Rating,
                'Property Description':property_description})
df